In [1]:
import pandas as pd
import numpy as np
import glob 
import docx2txt
from scipy import spatial
import gensim.downloader as api
import itertools

#Get list of all docx
list_of_docx = glob.glob('./documents/*.docx')
len(list_of_docx)

100

In [2]:
tuple_of_documents = []
for doc in list_of_docx:
    text = docx2txt.process(doc)
    tuple_of_documents.append((doc, text))

In [175]:
# model = api.load("word2vec-google-news-300") 
import nltk, string
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
#tokenizer
def removestopwords(tokens):
    return [item for item in tokens if item not in stop_words]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return removestopwords(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)) )

def get_vector(s):
    c = 0
    z = np.hstack(np.zeros(300))
    for idx, token in enumerate(normalize(s)):
    
        try:
            vector = model[token]
        except Exception:
            vector = np.zeros(300)
        
        z = np.hstack((z, vector))
    z = z.reshape(-1, 300)
    output = np.mean(z, axis = 0)
    return output
        
            
    return np.mean(np.array([model[i] for i in normalize(s)]), axis=0)

    

def get_spatial_similarity(s0, s1):
    return 1 - spatial.distance.cosine(get_vector(s0), get_vector(s1))

In [176]:
combinations = []
combination_iterator = itertools.combinations(tuple_of_documents, 2)
for i in combination_iterator:
    combinations.append({f'{i[0][0]} -> {i[1][0]}': [i[0][1], i[1][1]]})

In [177]:
from tqdm import tqdm
final = {}

for i in tqdm(combinations):
    item = list(i.items())[0]
    key = item[0]
    s0, s1 = item[1][0], item[1][1]

    final[key] = get_spatial_similarity(s0, s1)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4950/4950 [00:39<00:00, 125.97it/s]


In [178]:
output = pd.DataFrame.from_dict(final, orient='index')

In [181]:
output.to_csv('results.csv')